In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
snmahsa_animal_image_dataset_cats_dogs_and_foxes_path = kagglehub.dataset_download('snmahsa/animal-image-dataset-cats-dogs-and-foxes')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pathlib, cv2, os, copy, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from typing import Optional, Literal, Generator, Tuple, List, Dict
from matplotlib.patches import Wedge
from matplotlib.patheffects import withStroke
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

try:
    import splitfolders
except:
    ! pip install split-folders
    import splitfolders

import torch, torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.nn import Conv2d, ReLU, BatchNorm2d, Flatten, Linear, AvgPool2d, MaxPool2d, Dropout
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# GPU operations have a separate seed we also want to set
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
def get_dataframe_from_image_paths(image_folder_path: str) -> pd.DataFrame:
    """ Get images, labels and converts to dataframe

        Args:
            image_folder_path(str): Folders containing images
    """
    images = list(pathlib.Path(image_folder_path).glob("*/*"))
    labels = [os.path.split(os.path.dirname(img_path))[-1] for img_path in images]
    dataframe = pd.DataFrame(zip(images, labels), columns=["image_path", "labels"])
    dataframe["image_path"] = dataframe["image_path"].astype('str')
    return dataframe

def split_images_from_dataframe(dataframe: pd.DataFrame, val_split_size: int=0.3, test_split_size: int=0.1, split_test: bool=False)->tuple[list, list, list]:
    """ Splits dataframe into train, validation and test(optional)

        Args:
            dataframe(pd.Dataframe): Dataset Dataframe to split
            val_split_size(int): Validation split size
            test_split_size(int): Test split size
            split_test(bool): Flag to split validation further to test
    """
    train, val = train_test_split(dataframe, test_size=val_split_size, stratify=dataframe["labels"])
    test = None
    if split_test:
        val, test = train_test_split(val, test_size=test_split_size)
    return train, val, test

def plot_class_distribution_in_pie_chart(dataframe: pd.DataFrame):
    """ Plots dataset label distribution in pie chart

        Args:
            dataframe(pd.Dataframe): Dataset Dataframe to check class distribution
    """
    # Assuming your data is in a pandas DataFrame called 'train_df'
    animals_counts = dataframe['labels'].value_counts()

    fig, ax = plt.subplots()
    wedges, texts, _ = ax.pie(
        animals_counts.values.astype("float"), startangle=90,
        autopct='%1.1f%%', wedgeprops=dict(width=0.3, edgecolor='black')
    )

    # Add glow effect to each wedge
    for wedge in wedges:
        wedge.set_path_effects([withStroke(linewidth=6, foreground='cyan', alpha=0.4)])

    # Customize chart labels
    plt.legend(dataframe.index, loc="center left", bbox_to_anchor=(1, 0, 0.5, 1), fontsize=10, facecolor='#222222')

    # Dark background for the cyberpunk look
    fig.patch.set_facecolor('#2c2c2c')
    ax.set_facecolor('#2c2c2c')

    # Title
    plt.title("Pie Chart", color="white", fontsize=16)

    plt.show()

def plot_class_distribution_in_count_chart(dataframe: pd.DataFrame):
    """ Plots dataset label distribution in bar chart

        Args:
            dataframe(pd.Dataframe): Dataset Dataframe to check class distribution
    """
    #count Plot

    plt.figure(figsize=(8, 6))
    ax = sns.countplot(dataframe, x="labels", palette='pastel')

    # Annotate the count on top of each bar
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{int(height)}',
                    (p.get_x() + p.get_width() / 2, height),
                    ha='center', va='bottom')

    plt.tight_layout()
    # Show the plot
    plt.show()

def plot_random_images_per_class(dataframe: pd.DataFrame, no_images:int=2):
    """ Plots images per label distribution

        Args:
            dataframe(pd.Dataframe): Dataset Dataframe to check class distribution
    """
    class_names = dataframe["labels"].unique()
    random_images = dataframe.sample(frac=1).sort_values(by="labels").groupby('labels').head(no_images)

    count = 0
    num_classes = len(class_names)

    plt.figure(figsize=(12, num_classes * 4))

    for index, (image_path, class_name) in random_images.iterrows():
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        count += 1
        plt.subplot(num_classes, 2, count)
        plt.imshow(image)
        plt.axis('off')
        plt.title(class_name)

def plot_percentage_split(train_dataset: np.array, val_dataset: np.array, test_dataset: Optional[np.array]=None):
    """ Plots dataset split distribution in pie chart

        Args:
            train_dataset(np.array): Train dataset splitted
            val_dataset(np.array): Validation dataset splitted
        KwArgs:
            test_dataset(np.array): Test dataset splitted
    """
    train_size = len(train_dataset)
    validation_size = len(val_dataset)
    test_size = len(test_dataset or [])
    # Dataset sizes
    sizes = [train_size, validation_size]
    labels = ['Train', 'Validation']
    colors = ['#66c2a5', '#fc8d62']

    if test_dataset:
        sizes.append(test_size)
        labels.append("Test")
        colors.append("#0000ff")


    def autopct_format(value):
        """Formats the autopct value to display the percentage and count."""
        total = sum(sizes)
        percentage = f'{value:.1f}%'
        count = int(value * total / 100)
        return f'{percentage}\n{count}'

    # Create a pie chart
    plt.figure(figsize=(8, 8))
    plt.pie(sizes, labels=labels, colors=colors, autopct=autopct_format, startangle=140)
    plt.title('Dataset Split Distribution', fontsize=16)
    plt.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
    plt.show()


In [ ]:
def get_torch_transforms():
    """ Returns train and test transforms
    """
    train_transforms = A.Compose(
        transforms=[
            A.Resize(height=224, width=224),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
            A.RandomCrop(height=128, width=128),
            A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ]
    )

    val_transform = A.Compose(
        transforms=[
            A.Resize(height=224, width=224),
            A.CenterCrop(height=128, width=128),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ]
    )

    return train_transforms, val_transform


class CustomDataset(Dataset):
  """ Custom Dataset to apply transforms on dataframe

  Args:
    dataframe(pd.Dataframe): Dataset
    transforms(list): List of transforms to apply
  """
  def __init__(self, dataframe, transforms, class_to_idx):
    super().__init__()
    self._dataframe = dataframe
    self._transforms = transforms
    self._class_to_idx = class_to_idx

  def __len__(self):
    """ Returns length of dataframe
    """
    return len(self._dataframe)

  def __getitem__(self, index):
    """ Returns image and labels from specified index
    """
    image_path, label = self._dataframe.iloc[index]
    image = cv2.imread(image_path)
    print(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transformed_image = self._transforms(image=image)["image"]
    return transformed_image, self._class_to_idx[label]

def get_loaders(train_dataset, val_dataset, test_dataset, batch_size):
    """ Returns Torch dataloader for train, val and test
    """
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    return train_loader, val_loader, test_loader

def visualize_augmentations(dataset, idx=0, samples=10, cols=5):
    """ View the augmented images
    """
    dataset = copy.deepcopy(dataset)
    dataset._transforms = A.Compose([t for t in dataset._transforms.transforms if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i in range(samples):
        image, _ = dataset[idx]
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()

In [ ]:
# dataset creation to dataframe
image_folder = os.path.join("/kaggle/input/animal-image-dataset-cats-dogs-and-foxes", 'Animal Image Dataset-Cats, Dogs, and Foxes')
dataframe = get_dataframe_from_image_paths(image_folder)
class_names = dataframe["labels"].unique()

In [ ]:

# visualisation
plot_class_distribution_in_pie_chart(dataframe)
plot_class_distribution_in_count_chart(dataframe)
plot_random_images_per_class(dataframe)

In [ ]:
train_, val_, test_ = split_images_from_dataframe(dataframe, split_test=True)
train_transforms, val_transforms = get_torch_transforms()

class_to_idx = {cls_name: i for i, cls_name in enumerate(dataframe["labels"].unique())}
train_dataset = CustomDataset(train_, train_transforms, class_to_idx)
val_dataset = CustomDataset(val_, val_transforms, class_to_idx)
test_dataset = CustomDataset(test_, val_transforms, class_to_idx)

In [ ]:
train_loader, val_loader, test_loader = get_loaders(train_dataset, val_dataset, test_dataset, batch_size=32)
visualize_augmentations(train_dataset)

In [ ]:
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None):
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [ ]:

weights = torchvision.models.AlexNet_Weights.DEFAULT

# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torchvision.models.alexnet(weights=weights)
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.parameters():
    param.requires_grad = False

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=9216,
                    out_features=512, # same number of output units as our number of classes
                    bias=True),
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=512,
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)
model = model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=3)

writer = create_writer(
    experiment_name="Animal dataset",
    model_name="Alexnet",
    extra="learning animals dataset from torch api")

In [ ]:
# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

summary(model=model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        scheduler.step(loss.item())

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    writer: A SummaryWriter() instance to log model results to.

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc},
                               global_step=epoch)

            # Close the writer
            writer.close()

    # Return the filled results at the end of the epochs
    return results

In [ ]:
results = train(
    model, train_loader, test_loader, optimizer, loss_fn, 5, device, writer
)

In [ ]:
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    loss = results["train_loss"]
    test_loss = results["test_loss"]

    accuracy = results["train_acc"]
    test_accuracy = results["test_acc"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

In [ ]:
plot_loss_curves(results)

In [ ]:
def pred_and_plot_image(
    model: torch.nn.Module,
    class_names: List[str],
    image_path: str,
    target_image_act_label: str,
    image_size: Tuple[int, int] = (224, 224),
    transform: torchvision.transforms = None,
    device: torch.device = device,
):
    """Predicts on a target image with a target model.

    Args:
        model (torch.nn.Module): A trained (or untrained) PyTorch model to predict on an image.
        class_names (List[str]): A list of target classes to map predictions to.
        image_path (str): Filepath to target image to predict on.
        image_size (Tuple[int, int], optional): Size to transform target image to. Defaults to (224, 224).
        transform (torchvision.transforms, optional): Transform to perform on image. Defaults to None which uses ImageNet normalization.
        device (torch.device, optional): Target device to perform prediction on. Defaults to device.
    """

    # Open image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose(
            [
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    ### Predict on image ###

    # Make sure the model is on the target device
    model.to(device)

    # Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = image_transform(image=img)["image"].unsqueeze(dim=0)

        # Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(device))

    # Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # Plot image with predicted label and probability
    plt.figure()
    plt.imshow(img)
    plt.title(
        f"Pred: {class_names[target_image_pred_label]} | Actual: {target_image_act_label} | Prob: {target_image_pred_probs.max():.3f}"
    )
    plt.axis(False)

In [ ]:
image_path, label = test_.iloc[random.randint(0, len(test_))]
pred_and_plot_image(model, class_names, image_path, label, transform=val_transforms, device=device)